In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)   # 去掉第一层索引
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

# zero = sales_train[sales_train.date_block_num==0]
# ic(zero.shop_id.unique(), len(zero.item_id.unique()), len(zero.shop_id.unique()), len(zero.shop_id.unique()) * len(zero.item_id.unique()))
# ic(sales_train.shop_id.unique(), len(sales_train.item_id.unique()), len(sales_train.shop_id.unique()), len(sales_train.shop_id.unique()) * len(sales_train.item_id.unique()))

"""组合date_block_num,shop_id,item_id(部分) 总量：10913850"""
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

# 分组
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

# test数据
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

# 合并matrix,test
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix['item_cnt_month'].fillna(0, inplace=True)

# 商品信息
items = pd.read_csv('./data/items.csv')
items = items[['item_id', 'item_category_id']]
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并

# 商品类别
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并

# 商店信息
shops = pd.read_csv('./data/shops.csv')
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])

def st(name):
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt
shops['shop_type'] = shops['shop_name'].apply(st)

shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)


"""历史信息"""

def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)



# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商品大类） ++++++++++++ 和 月销量（商品）是重复的，因为每一个商品，类别是确定的，大类也是确定的
group = matrix.groupby(['date_block_num', 'item_id', 'cat_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_type_avg_item_cnt')
matrix.drop('date_item_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)



# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)] = (matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

# 月销量（商店类型）
group = matrix.groupby(['date_block_num','shop_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shoptype_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shoptype_avg_item_cnt')
matrix.drop('date_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店类型）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_shoptype_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_shoptype_avg_item_cnt')
matrix.drop('date_item_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店-商品）
group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shopitem_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shopitem_avg_item_cnt')
matrix.drop('date_shopitem_avg_item_cnt', axis=1, inplace=True)

# 趋势特征--item_cnt_month
group = matrix.groupby('item_id').agg({'item_cnt_month': 'mean'})
group.columns = ['trend_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['trend_date_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1, 2, 3, 4, 5, 6, 12]
matrix = lag_features(matrix, lags, 'trend_date_item_avg_cnt_month')
for i in lags:
    matrix['delta_cnt_month_lag_' + str(i)] = (matrix['trend_date_item_avg_cnt_month_lag_' + str(i)] - matrix[
        'trend_item_avg_cnt_month']) / matrix['trend_item_avg_cnt_month']


def select_trend2(row):
    for i in lags:
        if pd.notnull(row['delta_cnt_month_lag_' + str(i)]):  # 如果不是NaN
            return row['delta_cnt_month_lag_' + str(i)]
    return 0  # 如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势


matrix['delta_cnt_month_lag'] = matrix.apply(select_trend2, axis=1)

matrix['delta_cnt_month_lag'] = matrix['delta_cnt_month_lag'].astype(np.float16)

features_to_drop = ['trend_item_avg_cnt_month', 'trend_date_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['trend_date_item_avg_cnt_month_lag_' + str(i)]
    features_to_drop += ['delta_cnt_month_lag_' + str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)


# # 趋势特征 +++++++++++++++++++++++++++++++++++++
# group = matrix.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
# group.columns = ['qushi_shop_item_avg_cnt_month']
# group = group.reset_index()
# matrix = pd.merge(left=matrix, right=group, on=['shop_id', 'item_id'], how='left')

# group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
# group.columns = ['qushi_date_shop_item_avg_cnt_month']
# group = group.reset_index()
# matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')

# # 计算matrix中商品的历史价格
# lags = [1,2,3,4,5,6,12]
# matrix = lag_features(matrix, lags, 'qushi_date_shop_item_avg_cnt_month')
# for i in lags:
#     matrix['delta2_cnt_month_lag_'+str(i)] = (matrix['qushi_date_shop_item_avg_cnt_month_lag_' + str(i)] - matrix['qushi_shop_item_avg_cnt_month']) / matrix['qushi_shop_item_avg_cnt_month']

# def select_trend3(row):
#     for i in lags:
#         if pd.notnull(row['delta2_cnt_month_lag_'+str(i)]):  # 如果不是NaN
#             return row['delta2_cnt_month_lag_'+str(i)]
#     return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

# matrix['delta2_cnt_month_lag'] = matrix.apply(select_trend3, axis=1)
# matrix['delta2_cnt_month_lag'] = matrix['delta2_cnt_month_lag'].astype(np.float16)

# features_to_drop = ['qushi_shop_item_avg_cnt_month','qushi_date_shop_item_avg_cnt_month']
# for i in lags:
#     features_to_drop += ['delta2_cnt_month_lag_'+str(i)]
# matrix.drop(features_to_drop, axis=1, inplace=True)



In [2]:
matrix.shape

(11128050, 80)

In [3]:
# matrix_11128050_80_bak = matrix.copy()  

In [4]:
# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

In [5]:
# 找到有NaN值的列，然后把那些列中的NaN值填充0
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)

for i in column_null:
    matrix[i].fillna(0, inplace=True)

In [11]:
"""建模"""

trainData = matrix[matrix['date_block_num'] < 34]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 3000,
    'max_depth': 8,
    'num_leaves': 220,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.005,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.363340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12987
[LightGBM] [Info] Number of data points in the train set: 6425094, number of used features: 79
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.287729
[1]	training's rmse: 1.18397	valid_1's rmse: 1.13484
[2]	training's rmse: 1.18152	valid_1's rmse: 1.13299
[3]	training's rmse: 1.1792	valid_1's rmse: 1.13119
[4]	training's rmse: 1.17674	valid_1's rmse: 1.12939
[5]	training's rmse: 1.17419	valid_1's rmse: 1.12737
[6]	training's rmse: 1.17181	valid_1's rmse: 1.12565
[7]	training's rmse: 1.1699	valid_1's rmse: 1.12422
[8]	training's rmse: 1.16748	valid_1's rmse: 

[141]	training's rmse: 0.967432	valid_1's rmse: 0.979483
[142]	training's rmse: 0.966499	valid_1's rmse: 0.978967
[143]	training's rmse: 0.965505	valid_1's rmse: 0.978332
[144]	training's rmse: 0.964561	valid_1's rmse: 0.977695
[145]	training's rmse: 0.963625	valid_1's rmse: 0.977094
[146]	training's rmse: 0.962784	valid_1's rmse: 0.976511
[147]	training's rmse: 0.961939	valid_1's rmse: 0.97598
[148]	training's rmse: 0.961116	valid_1's rmse: 0.975467
[149]	training's rmse: 0.960275	valid_1's rmse: 0.974914
[150]	training's rmse: 0.959476	valid_1's rmse: 0.974412
[151]	training's rmse: 0.958628	valid_1's rmse: 0.973879
[152]	training's rmse: 0.957785	valid_1's rmse: 0.973313
[153]	training's rmse: 0.95698	valid_1's rmse: 0.972813
[154]	training's rmse: 0.955941	valid_1's rmse: 0.972016
[155]	training's rmse: 0.955044	valid_1's rmse: 0.971386
[156]	training's rmse: 0.954201	valid_1's rmse: 0.970825
[157]	training's rmse: 0.953199	valid_1's rmse: 0.970195
[158]	training's rmse: 0.952203	v

[281]	training's rmse: 0.881915	valid_1's rmse: 0.924398
[282]	training's rmse: 0.88144	valid_1's rmse: 0.924093
[283]	training's rmse: 0.881082	valid_1's rmse: 0.923922
[284]	training's rmse: 0.880739	valid_1's rmse: 0.923707
[285]	training's rmse: 0.880244	valid_1's rmse: 0.92335
[286]	training's rmse: 0.879893	valid_1's rmse: 0.923151
[287]	training's rmse: 0.879588	valid_1's rmse: 0.922982
[288]	training's rmse: 0.879267	valid_1's rmse: 0.922825
[289]	training's rmse: 0.878945	valid_1's rmse: 0.922654
[290]	training's rmse: 0.878496	valid_1's rmse: 0.922252
[291]	training's rmse: 0.878179	valid_1's rmse: 0.922097
[292]	training's rmse: 0.877763	valid_1's rmse: 0.921855
[293]	training's rmse: 0.877465	valid_1's rmse: 0.921708
[294]	training's rmse: 0.877188	valid_1's rmse: 0.921535
[295]	training's rmse: 0.876903	valid_1's rmse: 0.921402
[296]	training's rmse: 0.876596	valid_1's rmse: 0.921253
[297]	training's rmse: 0.876132	valid_1's rmse: 0.920954
[298]	training's rmse: 0.875787	v

[407]	training's rmse: 0.847683	valid_1's rmse: 0.903863
[408]	training's rmse: 0.847455	valid_1's rmse: 0.903729
[409]	training's rmse: 0.847286	valid_1's rmse: 0.903593
[410]	training's rmse: 0.847107	valid_1's rmse: 0.903515
[411]	training's rmse: 0.846956	valid_1's rmse: 0.903409
[412]	training's rmse: 0.846708	valid_1's rmse: 0.903248
[413]	training's rmse: 0.846482	valid_1's rmse: 0.903133
[414]	training's rmse: 0.846307	valid_1's rmse: 0.903023
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[415]	training's rmse: 0.846167	valid_1's rmse: 0.90297
[416]	training's rmse: 0.846013	valid_1's rmse: 0.902898
[417]	training's rmse: 0.8458	valid_1's rmse: 0.902732
[418]	training's rmse: 0.845611	valid_1's rmse: 0.902632
[419]	training's rmse: 0.845433	valid_1's rmse: 0.902535
[420]	training's rmse: 0.84528	valid_1's rmse: 0.902456
[421]	training's rmse: 0.845115	valid_1's rmse: 0.902373
[422]	training's rmse: 0.844949	valid_1's rmse: 0.902298
[LightGBM] [Warni

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[518]	training's rmse: 0.831026	valid_1's rmse: 0.894313
[519]	training's rmse: 0.83076	valid_1's rmse: 0.894075
[520]	training's rmse: 0.830638	valid_1's rmse: 0.894
[521]	training's rmse: 0.830563	valid_1's rmse: 0.893935
[522]	training's rmse: 0.830376	valid_1's rmse: 0.893834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[523]	training's rmse: 0.830143	valid_1's rmse: 0.893579
[524]	training's rmse: 0.830034	valid_1's rmse: 0.893492
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[525]	training's rmse: 0.829936	valid_1's rmse: 0.893406
[526]	training's rmse: 0.829715	valid_1's rmse: 0.893257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[527]	training's rmse: 0.829605	valid_1's rmse: 0.89323
[528]	training's rmse: 0.82941	valid_1's rmse: 0.892956
[529]	training's rmse: 0.829274	valid_1's rmse: 0.892892
[530]	training's rmse:

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2745]	training's rmse: 0.752948	valid_1's rmse: 0.836296
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2746]	training's rmse: 0.752937	valid_1's rmse: 0.836284
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2747]	training's rmse: 0.752925	valid_1's rmse: 0.836279
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2748]	training's rmse: 0.752906	valid_1's rmse: 0.836265
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2749]	training's rmse: 0.752898	valid_1's rmse: 0.836256
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2750]	training's rmse: 0.752887	valid_1's rmse: 0.836251
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2751]	training's rmse: 0.752878	valid_1's rmse: 0.836246
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2809]	training's rmse: 0.752126	valid_1's rmse: 0.835685
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2810]	training's rmse: 0.752114	valid_1's rmse: 0.83568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2811]	training's rmse: 0.752105	valid_1's rmse: 0.83567
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2812]	training's rmse: 0.752093	valid_1's rmse: 0.835667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2813]	training's rmse: 0.752081	valid_1's rmse: 0.835655
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2814]	training's rmse: 0.752071	valid_1's rmse: 0.835651
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2815]	training's rmse: 0.752056	valid_1's rmse: 0.835641
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2872]	training's rmse: 0.751335	valid_1's rmse: 0.834999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2873]	training's rmse: 0.75132	valid_1's rmse: 0.834993
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2874]	training's rmse: 0.751304	valid_1's rmse: 0.834983
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2875]	training's rmse: 0.751292	valid_1's rmse: 0.834968
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2876]	training's rmse: 0.75128	valid_1's rmse: 0.834963
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2877]	training's rmse: 0.751273	valid_1's rmse: 0.834941
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2878]	training's rmse: 0.751256	valid_1's rmse: 0.834934
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2936]	training's rmse: 0.750548	valid_1's rmse: 0.834406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2937]	training's rmse: 0.750535	valid_1's rmse: 0.834406
[2938]	training's rmse: 0.750515	valid_1's rmse: 0.834399
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2939]	training's rmse: 0.750509	valid_1's rmse: 0.834391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2940]	training's rmse: 0.7505	valid_1's rmse: 0.834385
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2941]	training's rmse: 0.750492	valid_1's rmse: 0.834382
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2942]	training's rmse: 0.75048	valid_1's rmse: 0.834374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2943]	training's rmse: 0.750473	valid_1's rmse: 0.834371
[LightGBM] [Wa

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3000]	training's rmse: 0.749854	valid_1's rmse: 0.833878


In [12]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})

test0 = test[test.item_id.isin(six_zero_item_id)]
ids = list(test0.ID.values)
submission.loc[submission.ID.isin(ids), 'item_cnt_month'] = 0.0
submission.to_csv('./submit/sub2.csv', index=False)